# Tutorial Part 1: Key Strings and Key Lists
Isopy supports a number of different kinds of geochemical data. Each type of data type has its own *flavour* of python object with unique attributes/methods useful for that particular data type. A key string is a subclass of ``str`` representing a certain flavour of data.  Isopy will automatically convert strings into the correct format so it is often not necessary for the user to strictly adhere to the key string format in normal usage. A collection of key strings is called a key list (Although key lists are in fact a subclass of tuple).

This introduction does not cover all the functionality of key strings and key lists. For a comprehensive overview the isopy docs [here](https://isopy.readthedocs.io/en/latest/refpages/dtypes.html#).

**Table of Content**

* [MassKeyString & MassKeyList](#MassKeyString-&-MassKeyList)
* [ElementKeyString & ElementKeyList](#ElementKeyString-&-ElementKeyList)
* [IsotopeKeyString & IsotopeKeyList](#IsotopeKeyString-&-IsotopeKeyList)
* [RatioKeyString & RatioKeyList](#RatioKeyString-&-RatioKeyList)
* [GeneralKeyString & GeneralKeyList](#GeneralKeyString-&-GeneralKeyList)
* [MixedKeyList](#MixedKeyList)
* [keystring & keylist](#keystring-&-keylist)
* [askeystring & askeylist](#askeystring-&-askeylist)
* [Turning key strings into python strings](#Turning-key-strings-into-python-strings)
* [Evaluating key strings](#Evaluating-key-strings)
* [Evaluating key lists](#Evaluating-key-lists)
* [Key list methods](#Key-list-methods)
* [Filtering key lists](#Filtering-key-lists)

In [1]:
# dont forget to import isopy
import isopy

## MassKeyString & MassKeyList
The ``Mass``  flavour represent data described by a mass number. Therefore key strings are restricted to integer numbers, e.g. ``"105"``. Key strings can be created from both integers and strings. **However**, when indexing an isopy array only a string can be used to represent the column as integer values represent row numbers.

In [2]:
isopy.MassKeyString('104'), isopy.MassKeyString(105)

(MassKeyString('104'), MassKeyString('105'))

In [3]:
isopy.MassKeyList('104', 105)

MassKeyList('104', '105')

## ElementKeyString & ElementKeyList
The ``Element``  flavour represents elemental data using the element symbol. Key string are restricted to one or two characters. The first character is always
in upper case and the second character, if present, is always in lower case, e.g. ``"Pd"``. Isopy will automatically format strings so they adhere to this format so any case can be used for element key strings. You can also use the full english names for the elements.

In [4]:
isopy.ElementKeyString('Pd'), isopy.ElementKeyString('Cadmium')

(ElementKeyString('Pd'), ElementKeyString('Cd'))

In [5]:
isopy.ElementKeyList('Pd', 'pd', 'PD', 'Cadmium', 'cadmium', 'CADMIUM')

ElementKeyList('Pd', 'Pd', 'Pd', 'Cd', 'Cd', 'Cd')

## IsotopeKeyString & IsotopeKeyList
The ``Isotope``  flavour represent isotope data and the key string consists of a ``Mass`` key string followed by an ``Element`` key string, e.g ``"105Pd"``. The order of the mass number and the element symbol is not enforced when creating isotope key strings. A ``-`` can be used to separate the mass number and the element symbol.

In [6]:
isopy.IsotopeKeyString('105pd'), isopy.IsotopeKeyString('pd105')

(IsotopeKeyString('105Pd'), IsotopeKeyString('105Pd'))

In [7]:
isopy.IsotopeKeyList('102Pd', '104pd', 'Pd105', '106Palladium', 'palladium-108', '110-PALLADIUM' )

IsotopeKeyList('102Pd', '104Pd', '105Pd', '106Pd', '108Pd', '110Pd')

---
The mass number and element symbol of a key string can be accessed by:

In [8]:
key = isopy.IsotopeKeyString('105pd')
key.mass_number, key.element_symbol

(MassKeyString('105'), ElementKeyString('Pd'))

Likewise for key lists:

In [9]:
keylist = isopy.IsotopeKeyList('102Pd', '104pd', 'Pd105', '106Palladium', 'palladium-108', '110-PALLADIUM' )
keylist.mass_numbers

MassKeyList('102', '104', '105', '106', '108', '110')

In [10]:
keylist.element_symbols

ElementKeyList('Pd', 'Pd', 'Pd', 'Pd', 'Pd', 'Pd')

## RatioKeyString & RatioKeyList

The ``Ratio``  flavour represents a ratio between two sets of data. The ratio key string consists of a numerator key string and a denominator key string, e.g. ``"108Pd/105Pd"``. The numerator and denominator can be of different flavours.

In [11]:
isopy.RatioKeyString('108pd/105pd')

RatioKeyString('108Pd/105Pd')

In [12]:
isopy.RatioKeyList('ru/pd', 'rh/pd', 'ag/pd', 'cd/pd')

RatioKeyList('Ru/Pd', 'Rh/Pd', 'Ag/Pd', 'Cd/Pd')

You can mix the flavours of the denominators and numerators in a ratio key list

In [13]:
isopy.RatioKeyList('ru/pd', '103rh/105pd', '111cd/pd')

RatioKeyList('Ru/Pd', '103Rh/105Pd', '111Cd/Pd')

---
The numerator and denominator key string can be accesed by:

In [14]:
key = isopy.RatioKeyString('108pd/105pd')
key.numerator, key.denominator

(IsotopeKeyString('108Pd'), IsotopeKeyString('105Pd'))

Likewise for lists:

In [15]:
keylist = isopy.RatioKeyList('ru/pd', 'rh/pd', 'ag/pd', 'cd/pd')
keylist.numerators

ElementKeyList('Ru', 'Rh', 'Ag', 'Cd')

In [16]:
keylist.denominators

ElementKeyList('Pd', 'Pd', 'Pd', 'Pd')

---
If a ratio key list has a common denominator this key string can be accessed using the ``common_denominator`` attribute. This attribute will be ``None`` if there is no common denominator.

In [17]:
isopy.RatioKeyList('ru/pd', 'rh/pd', 'ag/pd', 'cd/pd').common_denominator

ElementKeyString('Pd')

---
You can also create ratio key strings using the ``/`` operator:

In [18]:
isopy.IsotopeKeyString('108pd') / '105pd' #Only one of the strings have to be a key string

RatioKeyString('108Pd/105Pd')

In [19]:
isopy.ElementKeyList('ru', 'rh', 'ag', 'cd') / 'pd' # A single string will be used for all key strings in a list

RatioKeyList('Ru/Pd', 'Rh/Pd', 'Ag/Pd', 'Cd/Pd')

In [20]:
['ru', 'rh', 'ag', 'cd'] / isopy.ElementKeyString('pd') #This also works

RatioKeyList('Ru/Pd', 'Rh/Pd', 'Ag/Pd', 'Cd/Pd')

---
You can create nested ratio key strings using multiple '/' in a string

In [21]:
key = isopy.RatioKeyString('rh/ag//pd'); key

RatioKeyString('Rh/Ag//Pd')

In [22]:
key.numerator, key.denominator

(RatioKeyString('Rh/Ag'), ElementKeyString('Pd'))

**Note** It is only possible to create up to 9 nested ratios.

### GeneralKeyString & GeneralKeyList
The ``General``  flavour represent data that cannot be described by any of the other flavours. There are no formatting restrictions for these key strings so any string is valid.

In [23]:
isopy.GeneralKeyString('Hermione')

GeneralKeyString('Hermione')

In [24]:
isopy.GeneralKeyList('Harry', 'Ron', 'Hermione')

GeneralKeyList('Harry', 'Ron', 'Hermione')

## MixedKeyList
The ``Mixed`` flavour represet data that contains a mix of flavours. 

In [25]:
isopy.MixedKeyList('pd', 'cd', '101ru', '105pd')

MixedKeyList('Pd', 'Cd', '101Ru', '105Pd')

If we slice the list so that only one flavour of keys remain then the returned list is will be of that flavour.

In [26]:
keylist = isopy.MixedKeyList('pd', 'cd', '101ru', '105pd')
keylist[:2], keylist[2:]

(ElementKeyList('Pd', 'Cd'), IsotopeKeyList('101Ru', '105Pd'))

If we try to mix two different kinds of flavours we typically get a ``Mixed`` flavour in return

In [27]:
isopy.ElementKeyList('pd', 'cd') + isopy.IsotopeKeyList('101ru', '105pd')

MixedKeyList('Pd', 'Cd', '101Ru', '105Pd')

**Note** There is not a ``Mixed`` key string.

## keystring & askeystring
These functions will convert strings into one of the isopy key string flavours. They will first attempt to convert the string into a mass, element, isotope or ratio key string. If that fails a general key string is returned.

In [28]:
isopy.keystring('pd'), isopy.askeystring('105pd')

(ElementKeyString('Pd'), IsotopeKeyString('105Pd'))

The difference between the functions is that ``askeystring`` will not attempt to convert a string that is already a key string.

In [29]:
key = isopy.GeneralKeyString('pd')
isopy.keystring(key), isopy.askeystring(key)

(ElementKeyString('Pd'), GeneralKeyString('pd'))

## keylist & askeylist
These functions will attempt to convert the input into a key string list. They will first attempt to convert the input into a mass, element, isotope or ratio key list. If that fails a mixed key list is returned.

In [30]:
isopy.keylist('ru', 'pd', 'cd'), isopy.keylist('ru', '105pd', 'hermione')

(ElementKeyList('Ru', 'Pd', 'Cd'), MixedKeyList('Ru', '105Pd', 'hermione'))

The difference between the functions is that ``askeylist`` only takes a single input and will not attempt to convert the input if it is already a key list.

In [31]:
keylist = isopy.GeneralKeyList('Ruthenium', 'Palladium', 'Cadmium')
isopy.keylist(keylist), isopy.askeylist(keylist)

(ElementKeyList('Ru', 'Pd', 'Cd'),
 GeneralKeyList('Ruthenium', 'Palladium', 'Cadmium'))

If you want a general key list rather than a mixed key list should set the ``mix_flavours`` argument to ``False``

In [32]:
isopy.keylist('ru', '105pd', 'hermione'), isopy.keylist('ru', '105pd', 'hermione', mix_flavours=False)

(MixedKeyList('Ru', '105Pd', 'hermione'),
 GeneralKeyList('ru', '105pd', 'hermione'))

## Turning key strings into python strings
You can convert a key string back into a normal string using the ``str()`` function or the ``str()`` method of the key string.

In [33]:
key = isopy.keystring('pd')
str(key), key.str()

('Pd', 'Pd')

The ``str()`` method also allows you to format the returned string using a number of predefined keywords.

In [34]:
key.str('{Es}, {es}, {ES}, {Name}, {name}, {NAME}') # 'es' is the element symbol and 'name' the full name

'Pd, pd, PD, Palladium, palladium, PALLADIUM'

In [35]:
key.str('Name') #you can also pass a keyword directly

'Palladium'

Have a look at the key string documentation for a detailed explanation of the different format options for each key string flavour.

---
Similarly, key lists return a list of strings using the ``strlist()`` method.

In [36]:
keylist = isopy.keylist('ru', 'pd', 'cd')
keylist.strlist()

['Ru', 'Pd', 'Cd']

This method allows you to format the key strings using the same formatting options as the key string

In [37]:
keylist.strlist('{Es}, {Name}')

['Ru, Ruthenium', 'Pd, Palladium', 'Cd, Cadmium']

In [38]:
keylist.strlist('Name')

['Ruthenium', 'Palladium', 'Cadmium']

## Evaluating key strings
Any python string that can be converted into a key string with the same value will evaluate as ``True``, e.g.

In [39]:
isopy.keystring('105pd') == 'palladium-105'

True

**Note** However, comparing against another key string flavour will always return ``False`` even if the string value could be converted, e.g.

In [40]:
isopy.ElementKeyString('Pd') == isopy.GeneralKeyString('Pd')

False

---
Mass key strings allow you to use ``<, >, <=, >=`` to compare the mass number against other mass key strings or scalar values

In [41]:
mass = isopy.keystring(105)
mass > 100, mass < 105, mass >= 100.0, mass <= 105.0, mass > isopy.keystring('100')

(True, False, True, True, True)

---
You can test if a mass number or element symbol is part of a isotope key string using ``in``

In [42]:
key = isopy.keystring('105pd')
'palladium' in key, 105 in key

(True, True)

Similarly you can use ``in`` to test whether a string is the numerator of denominator of a ratio key string

In [43]:
key = isopy.keystring('108pd/105pd')
'108pd' in key, 'pd105' in key

(True, True)

## Evaluating key lists
Just like a normal list you can use ``in`` to test membership of a key list.

In [44]:
keylist = isopy.keylist('ru', 'pd', 'cd')
'palladium' in keylist, 'ag' in keylist

(True, False)

You can compare two key lists using ``==`` and ``!=``.

In [45]:
isopy.keylist(['ru', 'pd', 'cd']) == ['ru', 'pd', 'cd']

True

---
You can create a new extended key list using ``+``

In [46]:
isopy.keylist(['ru', 'pd', 'cd']) + ['rh', 'ag']

ElementKeyList('Ru', 'Pd', 'Cd', 'Rh', 'Ag')

Similarly you can remove items from key lists using ``-``

In [47]:
isopy.keylist(['ru', 'pd', 'cd']) - 'pd'

ElementKeyList('Ru', 'Cd')

---
You can do *and*, *or* and *xor* comparisons using ``&``, ``|`` and ``^``

In [48]:
isopy.keylist(['ru', 'pd', 'cd']) & ['pd', 'cd', 'te'] #returns keys in both lists

ElementKeyList('Pd', 'Cd')

In [49]:
isopy.keylist(['ru', 'pd', 'cd']) | ['pd', 'cd', 'te'] #returns keys in at least one list

ElementKeyList('Ru', 'Pd', 'Cd', 'Te')

In [50]:
isopy.keylist(['ru', 'pd', 'cd']) ^ ['pd', 'cd', 'te'] #returns key in one list but not the other

ElementKeyList('Ru', 'Te')

**Note** that ``+``, ``-``, ``&``, ``|`` and ``^`` all return a new key list.

## Key list methods
The ``count`` and ``index`` methods behaves as they do in a normal tuple

In [51]:
keylist = isopy.keylist('ru', 'pd', 'cd')

In [52]:
keylist.count('ru'), keylist.index('cadmium')

(1, 2)

---

You can check is a key list contains duplicate key string using the ``has_duplicates()`` method

In [53]:
keylist.has_duplicates()

False

---
You can reverse the order of the list using the ``reversed()`` method

In [54]:
isopy.keylist('ru', 'pd', 'cd').reversed()

ElementKeyList('Cd', 'Pd', 'Ru')

You can sort key list using the ``sorted()`` method. How the list is sorted depends on the flavour of the list (See documentation). Element key list are sorted by the atomic number of the element

In [55]:
isopy.keylist('U', 'H', 'Pd').sorted()

ElementKeyList('H', 'Pd', 'U')

Element symbols without atomic numers are sorted alphabetically at the end of the list

In [56]:
isopy.keylist('U', 'H', 'Pd', 'a', 'zz')

ElementKeyList('U', 'H', 'Pd', 'A', 'Zz')

**Note** that ``sorted()`` and ``reversed()`` functions will return a list and sorting done by key value

In [57]:
sorted(isopy.keylist('U', 'H', 'Pd'))

[ElementKeyString('H'), ElementKeyString('Pd'), ElementKeyString('U')]

---
The ``flatten()`` method when used on a ratio key list will return the numerators followed by the denominators in a single key string list.The flatten method exists for the other key list flavours but there it only return a copy of the list.

In [58]:
isopy.RatioKeyList('ru/pd', 'ag/rh').flatten()

ElementKeyList('Ru', 'Ag', 'Pd', 'Rh')

## Filtering key lists
The ``filter()`` method provides a way to simple yet powerful way to filter the contents of key string lists and return only certain keys. Using the ``key_eq`` and ``key_neq`` keyword you can return only those keystring that are equal to/not equal to the supplied value(s). If you supply more that one filter key word then only the keys that pass all filter are returned.

In [59]:
isopy.keylist('ru', 'pd', 'cd').filter(key_eq=['rh', 'pd', 'ag', 'cd'])

ElementKeyList('Pd', 'Cd')

In [60]:
isopy.keylist('ru', 'pd', 'cd').filter(key_neq=['rh', 'pd', 'ag', 'cd'])

ElementKeyList('Ru')

**Note** the ``filter()`` method always returns a new key list

---
Mass key lists can be filtered using *lt*, *le*, *gt*, and *ge* for ``<``, ``<=``, ``>``, and ``>=`` comparisons

In [61]:
isopy.keylist('101', 102, 103).filter(key_le=102)

MassKeyList('101', '102')

---
You can filter isotope key lists based in the mass number or element symbol using ``mass_number_<filter>`` and ``element_symbol_<filter>``

In [62]:
isopy.IsotopeKeyList('101ru', '105pd', '111cd').filter(mass_number_ge='105', element_symbol_eq=['rh', 'ag', 'cd'])

IsotopeKeyList('111Cd')

---
Ratio key lists can be filtered based on the numerator and denominator strings using ``numerator_<filter>`` and ``denominator_<filter>``. You can use any filter valid for the numerator/denominator key string flavour

In [63]:
key = isopy.RatioKeyList('104Ru/101Ru', '108Pd/105pd', 'ag107/109ag', '108Cd/111Cd')
key.filter(numerator_neq = '104ru', denominator_element_symbol_neq='ag')

RatioKeyList('108Pd/105Pd', '108Cd/111Cd')

---

You can also filter key lists based on the flavour of the key string. This is particularly useful for the ``Mixed`` flavour.

In [64]:
isopy.MixedKeyList('pd', 'cd', '101ru', '105pd').filter(flavour_eq='isotope')

IsotopeKeyList('101Ru', '105Pd')